In [7]:
#read data(exclude replies) 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk import SnowballStemmer
import pandas as pd

text = pd.read_csv('Jan6(excludes replies).csv')
print('The number of tweets (excludes replies):',len(text))
#Text cleaning
import gensim
import gensim.corpora as corpora
import re
stemmer = SnowballStemmer('english')
porter = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')
stop_words += ['cultured','clean','cultivated','meat']
b = []
for i,u in text.iterrows():
    a = []
    word =''
    for words in str(u['tweets']).split():
        if '@' not in words: #remove @users
            if '#' not in words:
                if 'http' not in words: #remove URLs
                    if'&amp' not in words: #remove symbol
                        words = words.lower()# lower form
                        words = re.sub(r'[^a-zA-Z]', ' ', words) #replace non-alphabets characters with space. From "can't" to "can t"
                        if len(words)>3:
                            word += (words+' ')
    doc = ''
    for token in word.split():
        if token not in stop_words:
            token = porter.stem(token) #root form
            doc += (token+' ')
    b.append(doc)
text['processed']=[i for i in b]

The number of tweets (excludes replies): 6501


In [26]:
#unigram
unigram = []
unigram_list = []
for index, i in text.iterrows():
    unigram=''
    for word in i['processed'].split():
        unigram+= word+' '
    unigram_list.append(unigram)


import guidedlda
import numpy as np
import gensim

seed_topic_list = [['environ','environment','carbon','climat','greenhous','emiss'], #Environmental Impact
                   ['plant','vegan','vegetarian','animal','without','slaughter',
                   'cruelti','welfar'],# Animal Welfare
                  ['antibiot','antimicrobi','resist','safeti','contamin','consumpt','popul',
                  'demand','secur','wast','industri','suppli','convent','regular','system'], #Food System
                  ['seafood','sea','ocean','fish','aquacultur','shrimp'],#Seafood
                  ['delici','tast','tender','textur','juici','test','chicken','duck','poultri','egg'],#Poultry
                  ['cell','stem','cultur','muscl','biolog','divis'], #Process
                  ['seri','fund','funder','rais','invest','investor','dollar','pound','announc', 
                  'happi','thrill','excit'], #Announcement
                  ['thank','thankyou','support','shoutout','help'], #Appreciation
                  ['confer','regist','live','symposium','stream','livestream'], #conference and summit
                  ['market','consum','store','groceri'], # Market
                  ['check','post','paper','interview','articl','blog','news','break'], #Media
                  ['hire','join','team']]#Hiring Information

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
model = guidedlda.GuidedLDA(n_topics=12,n_iter=200,random_state=1,refresh=10,alpha=0.1,eta=0.01)
vectorizer = CountVectorizer(min_df = 2)
X = vectorizer.fit_transform(text['processed'])

vocab = vectorizer.get_feature_names()
word2id = dict((v,idx) for idx,v in enumerate(vocab))
seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
    for word in st:
        seed_topics[word2id[word]] = t_id

model.fit(X.toarray(),seed_topics=seed_topics,seed_confidence=0.35)
topic_word = model.topic_word_
n_top_words = 30
vocab = tuple(vocab)

for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

X = X.toarray()
doc_topic = model.transform(X)
topic_number = []
number = []
for i in range(len(doc_topic)-1):
    topic_number.append(doc_topic[i].argmax())
    number.append('1')
data = pd.DataFrame(data=[i for i in topic_number],columns=['topic_number'])
data['number'] = [i for i in number]
print(data.groupby('topic_number')['number'].count())

INFO:guidedlda:n_documents: 6501
INFO:guidedlda:vocab_size: 3621
INFO:guidedlda:n_words: 60068
INFO:guidedlda:n_topics: 12
INFO:guidedlda:n_iter: 200
INFO:guidedlda:<0> log likelihood: -694392
INFO:guidedlda:<10> log likelihood: -490422
INFO:guidedlda:<20> log likelihood: -474945
INFO:guidedlda:<30> log likelihood: -467499
INFO:guidedlda:<40> log likelihood: -463902
INFO:guidedlda:<50> log likelihood: -461861
INFO:guidedlda:<60> log likelihood: -460037
INFO:guidedlda:<70> log likelihood: -458819
INFO:guidedlda:<80> log likelihood: -457591
INFO:guidedlda:<90> log likelihood: -456854
INFO:guidedlda:<100> log likelihood: -456434
INFO:guidedlda:<110> log likelihood: -455634
INFO:guidedlda:<120> log likelihood: -455670
INFO:guidedlda:<130> log likelihood: -455698
INFO:guidedlda:<140> log likelihood: -455183
INFO:guidedlda:<150> log likelihood: -455020
INFO:guidedlda:<160> log likelihood: -454832
INFO:guidedlda:<170> log likelihood: -454291
INFO:guidedlda:<180> log likelihood: -454387
INFO:g

Topic 0: chang food anim world climat need sustain human solut health peopl produc environment global impact system agricultur reduc industri grow feed use planet eat water product earth problem help would
Topic 1: anim make without world real better free slaughter futur help want need grow planet peopl food save kill love produc harm support vegan human check chang cow billion cruelti word
Topic 2: food product plant industri market base protein global scale suppli demand chain system process could altern sustain cubiq consum toward like sale opportun investor anim commerci compani facil next report
Topic 3: fish seafood shrimp ocean cell sustain year grow salmon tast product like bluenalu market know aquacultur cellular wild provid delici number healthi way also around chef farm consum differ plastic
Topic 4: chicken tast first burger like base make cell product look made come plant good steak world anim meatbal bring pork delici cook great duck wait test grown market lobster real
To